# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

2つのモデルを用いて、入力されたシーケンスに基づいた別のシーケンスを出力するモデル。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings; warnings.filterwarnings('ignore')
import random

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 発想

これまで、RNNを用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できそう。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプションなどが例に挙げられる。  
学習方法は簡単で、入力画像に対して適切な文章が出力されるように学習させるだけ。隠れ状態を通じてRNNからCNNまで逆伝播を繋げる。

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。  

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。  
こういった、シーケンスをシーケンスに変換するモデルを *seq2seq (Sequence to Sequence)* と呼ぶ。

本章では、入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。

このモデルは以下の二つのRNNから構成される。
- ***Encoder*** : 文章ベクトルを生成するRNN
- ***Decoder*** : 文章ベクトルを受け取って出力文を生成するRNN

このことから***Encoder-Decoderモデル***とも呼ばれる。


---

## 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットを使用する。
- [iwslt2017  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/community_catalog/huggingface/iwslt2017?hl=ja#iwslt2017-en-ja)

In [3]:
ds = tfds.load(
    'huggingface:iwslt2017/iwslt2017-en-ja',
    data_dir='data',
    split='train'
)
ds = list(ds.as_numpy_iterator())

In [4]:
data_ja = []
data_en = []
for sample in ds:
    ja = sample['translation']['ja'].decode()
    en = sample['translation']['en'].decode()
    data_ja.append(ja)
    data_en.append(en)

print('num of data:', len(data_ja), '\n')
for _ in range(5):
    i = random.randint(0, len(data_ja))
    print(data_ja[i])
    print(data_en[i])
    print()

num of data: 223108 

送金の媒体を利用して このような債券を移民に販売できます 毎月の送金のタイミングに合わせて 毎月の送金のタイミングに合わせて 債券を売ることができます
Remittance channels can be used to sell these bonds to migrants because when they come on a monthly basis to send remittances, that's when you can actually sell it to them.

現在のロボットでは 真似できない
like no robot we have yet.

いつの日か 顕微鏡をいくつもいくつも並べて 全てのニューロンとシナプスを捉えた 巨大なイメージデータベースを作り
Someday, a fleet of microscopes will capture every neuron and every synapse in a vast database of images.

向こうでは、180パーセントの税金をガソリン車にかけ 排気ガスゼロの車に対しては、税金をかけません
They put 180 percent tax on gasoline cars and zero tax on zero-emission cars.

自然な発達過程には不適切であり とりわけ少年に悪影響を与えます
It's not developmentally appropriate, and it's particularly bad for boys.



書き出し

In [5]:
textfile_ja = 'data/iwslt2017_ja.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = 'data/iwslt2017_en.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

データ数を減らしたものも作っておく。

In [6]:
n_data = 10000
data_ja = data_ja[:n_data]
data_en = data_en[:n_data]

textfile_ja = f'data/iwslt2017_ja_{n_data}.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = f'data/iwslt2017_en_{n_data}.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

トークナイザの学習。

In [7]:
pad_id = 3
vocab_size = 8000

tokenizer_prefix_ja = f'models/tokenizer_iwslt2017_ja_{n_data}'
spm.SentencePieceTrainer.Train(
    input=textfile_ja,
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size,
    pad_id=pad_id
)

tokenizer_prefix_en = f'models/tokenizer_iwslt2017_en_{n_data}'
spm.SentencePieceTrainer.Train(
    input=textfile_en,
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size,
    pad_id=pad_id
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/iwslt2017_ja_10000.txt
  input_format: 
  model_prefix: models/tokenizer_iwslt2017_ja_10000
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privac

### 前処理

データ数を減らした方を使う。

トークナイザ読み込み

In [8]:
sp_ja = spm.SentencePieceProcessor(f'{tokenizer_prefix_ja}.model')
sp_en = spm.SentencePieceProcessor(f'{tokenizer_prefix_en}.model')

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()

n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print('num of vocabrary (ja):', n_vocab_ja)
print('num of vocabrary (en):', n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [9]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [10]:
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 学習データの作成

入力文と正解のペアを作成する。

Encoderへの入力（入力文）とDecoderの出力（正解）のペアを作成する。  
また、Decoderへの入力を考える必要がある。今回は教師強制を採用し、出力文の頭に\<BOS>を付与したものをDecoderへの入力とする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Eecoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

DataLoaderの作成。

In [11]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # encoderへの入力
        x_dec = en[:-1] # decoderへの入力
        y_dec = en[1:] # decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

(torch.Size([32, 64]), torch.Size([32, 68]), torch.Size([32, 68]))


---

## モデル構築

EncoderとDecoderを作る。

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN(LSTM)。LSTMと線形層で作る。

ここで1つ工夫を加える。普通に作っても動きはするが、学習が上手くいかない可能性がある。

encoderへの入力はpaddingされたデータである。ここで、paddingされた範囲が多い=padトークンが多く含まれているデータは、padの数が多くなるにつれて、隠れ状態がある一定の値に収束してしまう。RNNに同じトークンを何度も入力することで隠れ状態が収束していってしまうのだ。  
そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、翻訳を学習できない。

そこで、encoderが出力する隠れ状態として、padトークンを除いた最後のトークン=EOSを入力した時点のものを使用する。  

以上を踏まえ、encoderを以下のように実装する。

In [30]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size, dropout_rate=0.5):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_positions = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, _ = self.lstm(x) # (batch_size, seq_len, hidden_size)
        hs = self.dropout(hs)
        h = hs[eos_positions].unsqueeze(0) # (1, batch_size, hidden_size)
        return h

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、LSTMと線形層で作る。

In [31]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size, dropout_rate=0.5):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, hc):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, (h, c) = self.lstm(x, hc)
            # hs: (batch_size, seq_len, hidden_size)
            # h: (1, batch_size, hidden_size)
            # c: (1, batch_size, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (batch_size, seq_len, n_vocab)
        return y, (h, c)

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [32]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        hc = (h, torch.zeros_like(h))
        y, _ = self.decoder(x_dec, hc)
        return y

### 学習

In [33]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

def eval_model(model):
    model.eval()
    losses = []
    with torch.no_grad():
        for x_enc, x_dec, y_dec in test_loader:
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)
            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            losses.append(loss.item())
    loss = sum(losses) / len(losses)
    return loss

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_loss = eval_model(model)
            prog.memo(f'test: {test_loss:.5f}', no_step=True)
        prog.memo()

In [34]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [35]:
train(model, optimizer, n_epochs=100, prog_unit=10)

   1-10/100: ############################## 100% [00:01:06.61] loss train: 5.51102, test: 5.15894 
  11-20/100: ############################## 100% [00:01:20.69] loss train: 4.72939, test: 4.96114 
  21-30/100: ############################## 100% [00:01:13.05] loss train: 4.28014, test: 4.96832 
  31-40/100: ############################## 100% [00:01:23.97] loss train: 3.89920, test: 5.08351 
  41-50/100: ############################## 100% [00:01:24.39] loss train: 3.55994, test: 5.24198 
  51-60/100: ############################## 100% [00:01:17.29] loss train: 3.25648, test: 5.43300 
  61-70/100: #                                4% [00:00:03.33] loss train: 3.08988               

KeyboardInterrupt: 


---

## 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。

In [ ]:
def token_sampling(y, decisive=True):
    y.squeeze_(0)
    if decisive:
        token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device)

    h = model.encoder(in_ids)
    hc = (h, torch.zeros_like(h))
    next_token = bos_id

    token_ids = []
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([[next_token]], device=device)
        y, hc = model.decoder(x, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

まずは訓練データから。

In [ ]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: 彼らがバーミンガムの 少年少女十字軍について テレビで見たり聞いたりするのは 私達が1863年のリンカーンを 映画で観るのと似ています 歴史なのです
output: And so for them, when they hear about the Children's Crusade in Birmingham, in many ways, if they see it on TV, it's like our looking at the 1863 "Lincoln" movie: It's history.
answer: And so for them, when they hear about the Children's Crusade in Birmingham, in many ways, if they see it on TV, it's like our looking at the 1863 "Lincoln" movie: It's history.

input: 一番大切なのは menschであること」 どうもありがとうございました
output: The most important thing is to do it: not all the time right now.
answer: The most important thing is to be a mensch." Thank you.

input: 食べ物みたいに直接的対象ではありません 子供や恋人というのも
output: It would hardly do to eat your baby or your lover.
answer: It would hardly do to eat your baby or your lover.

input: ありがとうございました
output: Thank you.
answer: Thank you.

input: そう考えると ジャーナリストになった自分も含めた皆が 問題の一部であると 私は気づきました
output: It was when I became a journalist that I really realized how I was part of this problem, 

一致。

訓練データに含まれていないものも試してみる。

In [ ]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: このような指数的成長の 一貫性は
output: It's a real delicate machine running for example.
answer: It's pretty remarkable how smooth an exponential process that is.

input: 本日は 2つの プロジェクトを紹介します まずは最初のプロジェクトです
output: This Syrian young man survived one of these boats an up ⁇ 
answer: I'm going to share with you two projects that are investigations along these lines, and we'll start with this one.

input: 何がすごいかと言うと アメリカで公民権が獲得されたのは 60年代だったのです
output: So, this is the website of an indignity the machineary political world, and you know what's about data.
answer: Now, what is remarkable is that civil rights in America were achieved in the 1960s.

input: 私たちは地球への負担をより軽くする 術を学んでいくだろうと 確信しています 新しいデジタルツールによって起こることが とても大規模で非常に有益で それ以前のすべてのものが お笑い草になるほどのものだと
output: And if you look at honeybees, and I have GPS -- after I'm sure some of you have in the same way to show you pretty much more'tly doing, but supportive for one time, I think most of the best definition good is better good. The good life is com

In [ ]:
# original
sentences = [
    'ありがとう。',
    '猫はかわいいね。',
    '上手く文章が書けるようになりました。'
]

for sentence in sentences:
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print()

input: ありがとう。
output: Thank you.

input: 猫はかわいいね。
output: But it's not a very simple solution. The story with Ry.

input: 上手く文章が書けるようになりました。
output: They were built to transmit.



びみょう。